In [1]:
import torch
import torch.nn as nn


In [1]:

class Model(nn.Module):
    def __init__(self, n_input, n_context, n_hidden_1, n_hidden_2, n_hidden_3, n_cell_dim, n_hidden_5, n_hidden_6, n_classes, dropout_rate, layer_norm):
        super(Model, self).__init__()
        self.n_input = n_input
        self.n_context = n_context
        self.n_hidden_1 = n_hidden_1
        self.n_hidden_2 = n_hidden_2
        self.n_hidden_3 = n_hidden_3
        self.n_cell_dim = n_cell_dim
        self.n_hidden_5 = n_hidden_5
        self.n_hidden_6 = n_hidden_6
        self.n_classes = n_classes
        self.dropout_rate = dropout_rate
        self.layer_norm = layer_norm
        
        # Define layers
        self.layer_1 = nn.Linear(n_input + 2 * n_input * n_context, n_hidden_1)
        self.layer_2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.layer_3 = nn.Linear(n_hidden_2, n_hidden_3)
        self.rnn = nn.LSTM(n_hidden_3, n_cell_dim, batch_first=True)
        self.layer_5 = nn.Linear(n_cell_dim, n_hidden_5)
        self.layer_6 = nn.Linear(n_hidden_5, n_hidden_6)
        
    def forward(self, batch_x):
        batch_size = batch_x.size(0)
        
        # Reshape input
        batch_x = batch_x.permute(1, 0, 2, 3).contiguous()
        batch_x = batch_x.view(-1, self.n_input + 2 * self.n_input * self.n_context)
        
        # Hidden layers
        layer_1 = nn.functional.relu(self.layer_1(batch_x))
        layer_2 = nn.functional.relu(self.layer_2(layer_1))
        layer_3 = nn.functional.relu(self.layer_3(layer_2))
        
        # Reshape for RNN
        layer_3 = layer_3.view(-1, batch_size, self.n_hidden_3)
        
        # RNN layer
        output, (h_n, c_n) = self.rnn(layer_3)
        
        # Reshape output
        output = output.view(-1, self.n_cell_dim)
        
        # Hidden layer 5
        layer_5 = nn.functional.relu(self.layer_5(output))
        
        # Output layer
        raw_logits = self.layer_6(layer_5)
        
        # Reshape output
        raw_logits = raw_logits.view(-1, batch_size, self.n_hidden_6)
        
        return raw_logits

NameError: name 'nn' is not defined